In [6]:
import numpy as np
import math
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import KFold, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import accuracy_score, average_precision_score, precision_score,recall_score,roc_curve,precision_recall_curve, auc,f1_score, confusion_matrix
import operator
from scipy.optimize import fmin_bfgs
from sklearn.base import clone
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report

In [7]:
from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *
mnist = fetch_mldata('MNIST original')
mnist.data.shape
mnist.target.shape
np.unique(mnist.target)
X_m, y_m = mnist.data / 255., mnist.target
X_train, X_test = X_m[:1000], X_m[1000:]
y_train, y_test = y_m[:1000], y_m[1000:]
X_train.shape
y_train.shape
size=len(y_train)

In [8]:
X_train_m= X_test
Y_train_m= y_test.astype(np.int);
X_test_m=X_train
Y_test_m=y_train.astype(np.int);
print X_train_m.shape
print Y_train_m.shape
print set(Y_train_m)

(69000L, 784L)
(69000L,)
set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [9]:
def read_input(filename):
    input_data=[];
    f=open(filename);
    #f.readline();
    #f.readline();
    for line in f:
        input_data.append(line.rstrip('\n').split(','));
    return input_data;

def create_feature_matrix(input_data):
    x_list=[a[0:len(input_data[0])-1] for a in input_data];
    x=np.array([a[0:len(input_data[0])-1] for a in input_data]);
    #x=np.transpose(x)
    x= x.astype(np.float);
    return x

def create_y_matrix(input_data):
    y_list=[a[len(input_data[0])-1] for a in input_data];
    y_list_1=[];
    for i in y_list:
        if(i=='Iris-setosa'):
            y_list_1.append(0);
        elif(i=='Iris-versicolor'):
            y_list_1.append(1);
        elif(i=='Iris-virginica'):
            y_list_1.append(2);
        
            
    y=np.array(y_list_1);
    #y=np.transpose(y)
    y= y.astype(np.int);
    return y

In [10]:
#input_data=read_input('winequality-red.csv')
input_data=read_input('iris.dat')
X=create_feature_matrix(input_data)
Y=create_y_matrix(input_data)
print X.shape
print Y.shape

(150L, 4L)
(150L,)


In [11]:
def loss(x,y,model):
    w1,b1,w2,b2=model['w1'],model['b1'],  model['w2'], model['b2']
    fp1=np.dot(x,w1 )+b1;
    a1=np.tanh(fp1)
    fp2=np.dot(a1, w2)+b2;
    exp=np.exp(fp2)
    p=exp/np.sum(exp, axis=1, keepdims=True)
    l_theta=-np.log(p[range(x.shape[0]),y])
    data=np.sum(l_theta)
    data = data+0.01/2 * (np.sum(np.square(w1)) + np.sum(np.square(w2)))
    return 1./x.shape[0]*data

In [12]:
def predict(model,x):
    w1,b1,w2,b2=model['w1'],model['b1'],  model['w2'], model['b2']
    fp1=np.dot(x,w1 )+b1;
    a1=np.tanh(fp1)
    fp2=np.dot(a1, w2)+b2;
    exp=np.exp(fp2)
    p=exp/np.sum(exp, axis=1, keepdims=True)
    #print p
    return np.argmax(p, axis=1)

In [13]:
def network(x,y,hidden_layer, passes, verbose):
    w1= np.random.randn(x.shape[1], hidden_layer)/np.sqrt(x.shape[1]);
    w2= np.random.randn(hidden_layer,len(set(y)))/np.sqrt(hidden_layer);
    b1 = np.zeros((1, hidden_layer))
    b2 = np.zeros((1, len(set(y))))
    
    model={}
    for i in xrange(0, passes):
        fp1=np.dot(x, w1);
        a1=np.tanh(fp1);
        fp2=np.dot(a1, w2)
        exp=np.exp(fp2)
        #print exp.shape
        p=exp/np.sum(exp, axis=1, keepdims=True)
        d3=p;
        #print d3.shape
        d3[range(x.shape[0]), y]=d3[range(x.shape[0]), y]-1;
        dw1=np.dot(np.transpose(a1), d3)
        db1=np.sum(d3, axis=0, keepdims=True)
        
        d2=np.dot(d3, np.transpose(w2))*(1-np.power(a1,2));
        dw11=np.dot(np.transpose(x), d2);
        db11=np.sum(d2, axis=0)
        dw11 += 0.1 * w1
        dw1 += 0.1 * w2
        w1=w1-0.01*dw11;
        w2=w2-0.01*dw1;
        b1=b1-0.01*db11;
        b2=b2-0.01*db1;
        model={'w1':w1, 'b1': b1, 'w2': w2, 'b2':b2}
        if(verbose == True and i%1000 ==0):
            print "Loss: %f " %loss(x,y,model);
        return model;
        
    

In [46]:
def do_cross_validation(X, y,  classarray,n_folds, verbose=False):
    
    accuracies = []
    precisions =[]
    recalls=[]
    f1_scores=[]
    y_p=[]
    i=1;
    X1=X;
    Y1=y;    
    #Y1 = np.zeros((X1.shape[0], len(classarray)), dtype=np.float64)
    #for i, a in enumerate(classarray):
    #    Y1[y==a, i]=1;
    cv = KFold(len(Y1), n_folds, shuffle=True, random_state=5)
    #cv = StratifiedKFold(Y1, n_folds, shuffle=True, random_state=5)
    for train_idx, test_idx in cv:
        #theta=calc_theta(X1[train_idx],Y1[train_idx]);
        #theta=theta[0].reshape((X1[train_idx].shape[1], Y1[train_idx].shape[1]))
        #print theta
        #predicted_y=predict(X1[test_idx], Y1[test_idx], theta, classarray);
        #print Y1[test_idx];
        #print predicted_y;
        model=network(X[train_idx], Y1[train_idx], 4, 200000000, True)
        predicted_y=predict(model,X1[test_idx])
        #print predicted_y
        #print Y1[test_idx]
        #print accuracy_score(Y1[test_idx], predicted_y)
        acc = accuracy_score(Y1[test_idx], predicted_y)
        c_matrix= confusion_matrix(Y1[test_idx], predicted_y)
        precision=precision_score(Y1[test_idx], predicted_y, average="weighted")
        recall=recall_score(Y1[test_idx], predicted_y, average="weighted")
        f_score=f1_score(Y1[test_idx], predicted_y, average="weighted")
        if(verbose == True):
            print('fold %d accuracy=%.4f, precision=%.4f, recall=%.4f, F-measure%.4f, confusion matrix=' % (i, acc, precision, recall, f_score));
            print c_matrix
        i=i+1;
        accuracies.append(acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f_score)
        prredicted_y=[]
    avg=np.max(accuracies);
    avg_precision=np.max(precisions)
    avg_recall=np.max(recalls)
    avg_f1_score=np.max(f1_scores)
    return avg, avg_precision, avg_recall, avg_f1_score;

In [98]:
do_cross_validation(X, Y,[1,2,3],10, verbose=True)

Loss: 1.561323 
fold 1 accuracy=0.3333, precision=0.1111, recall=0.3333, F-measure0.1667, confusion matrix=
[[0 4 0]
 [0 5 0]
 [0 6 0]]
Loss: 1.308266 
fold 2 accuracy=0.2667, precision=0.0711, recall=0.2667, F-measure0.1123, confusion matrix=
[[4 0 0]
 [6 0 0]
 [5 0 0]]
Loss: 1.183273 
fold 3 accuracy=0.4667, precision=0.2178, recall=0.4667, F-measure0.2970, confusion matrix=
[[7 0 0]
 [5 0 0]
 [3 0 0]]
Loss: 1.389515 
fold 4 accuracy=0.3333, precision=0.1111, recall=0.3333, F-measure0.1667, confusion matrix=
[[0 5 0]
 [0 5 0]
 [0 5 0]]
Loss: 1.340252 
fold 5 accuracy=0.3333, precision=0.1111, recall=0.3333, F-measure0.1667, confusion matrix=
[[0 0 2]
 [0 0 8]
 [0 0 5]]
Loss: 1.117087 
fold 6 accuracy=0.2667, precision=0.0711, recall=0.2667, F-measure0.1123, confusion matrix=
[[0 7 0]
 [0 4 0]
 [0 4 0]]
Loss: 1.184173 
fold 7 accuracy=0.2667, precision=0.0711, recall=0.2667, F-measure0.1123, confusion matrix=
[[0 0 5]
 [0 0 6]
 [0 0 4]]
Loss: 0.954020 
fold 8 accuracy=0.3333, precisio

(0.8666666666666667,
 0.76000000000000001,
 0.8666666666666667,
 0.80740740740740735)

In [74]:
model=network(X, Y, 4, 500000000, True)
predicted_y= predict(model,X)
print "Accuracy: %f" %accuracy_score(Y, predicted_y)
print "Precision: %f" %precision_score(Y, predicted_y, average="weighted")
print "Recall: %f" %recall_score(Y, predicted_y, average="weighted")
print "F1 Scpre: %f" %f1_score(Y, predicted_y, average="weighted")
print "Confusion Matrix:"
print confusion_matrix(Y, predicted_y)

Loss: 0.992423 
Accuracy: 0.666667
Precision: 0.495148
Recall: 0.666667
F1 Scpre: 0.553747
Confusion Matrix:
[[50  0  0]
 [ 1  0 49]
 [ 0  0 50]]


In [100]:
do_cross_validation(X_train_m, Y_train_m,[0,1,2,3,4,5,6,7,8,9],10, verbose=True)

Loss: 19.348975 
fold 1 accuracy=0.0970, precision=0.0095, recall=0.0970, F-measure0.0173, confusion matrix=
[[  0  10   0   0   0   0   0   0   0 583]
 [  0   0   0   0   0   0   0   0   0 807]
 [  0   0   0   0   0   0   0   0   0 719]
 [  0   0   0   0   0   0   0   0   0 701]
 [  0  23   0   0   0   0   0   0   0 694]
 [  0   2   0   0   0   0   0   0   0 649]
 [  0   5   0   0   0   0   0   0   0 642]
 [  0   0   0   0   0   0   0   0   0 721]
 [  0   1   0   0   0   0   0   0   0 671]
 [  0   3   0   0   0   0   0   0   0 669]]
Loss: 26.694465 
fold 2 accuracy=0.0212, precision=0.0141, recall=0.0212, F-measure0.0083, confusion matrix=
[[  0   8   3   0   2   0 436 159   0   0]
 [  0   0   0   0  11   0 610 143   0   0]
 [  0   6   0   0   4   0 276 433   0   0]
 [  0   2   0   0   0   0 623  60   0   0]
 [  0   0 118   0   4   0   1 553   0   0]
 [  0   1   0   0   6   0 272 387   0   0]
 [  0   1   4   0   1   0   4 682   0   0]
 [  0 110  45   0   4   0 461 138   0   0]
 [  0  

(0.21231884057971015,
 0.13635509293144182,
 0.21231884057971015,
 0.14532333917348036)

In [161]:
model=network(X_train_m, Y_train_m, 4, 500000000, True)
predicted_y= predict(model,X_train_m)
print "Accuracy: %f" %accuracy_score(Y_train_m, predicted_y)
print "Precision: %f" %precision_score(Y_train_m, predicted_y, average="weighted")
print "Recall: %f" %recall_score(Y_train_m, predicted_y, average="weighted")
print "F1 Scpre: %f" %f1_score(Y_train_m, predicted_y, average="weighted")
print "Confusion Matrix:"
print confusion_matrix(Y_train_m, predicted_y)

Loss: 33.666639 
Accuracy: 0.177203
Precision: 0.141450
Recall: 0.177203
F1 Scpre: 0.109950
Confusion Matrix:
[[  11    5    0 5399    0  488    0    0    0    0]
 [   3 6758    0 1110    0    6    0    0    0    0]
 [  61  504    0 6227    0  198    0    0    0    0]
 [  45 1733    0 5252    0  110    0    1    0    0]
 [2593   33    0  662    0 3536    0    0    0    0]
 [ 138  415    0 5555    0  205    0    0    0    0]
 [  25   48    0 6346    0  457    0    0    0    0]
 [6063  324    0  295    0  610    0    1    0    0]
 [  28  687    0 5793    0  317    0    0    0    0]
 [1732  192    0  601    0 4433    0    0    0    0]]
